# Replicating analysis and data file from Neal & Johnson 1996

This page replicates the data analysis and data file preparation from the paper:

Neal, D. A., & Johnson, W. R. (1996). [The role of premarket factors in
black-white wage
differences](https://www.ssc.wisc.edu/~gwallace/Papers/Neal%20and%20Johnson%20%281996%29.pdf).
Journal of political Economy, 104(5), 869-895.

Professor Neal kindly provided me (MB) a Stata file `RECREATE.DO`, as well as a data file to check against, called `JPE96.DTA`.

See the `README.md` file in this directory for instructions to download the data files here of filename pattern `neal_johnson_nls*.*`.

The original version of this notebook checked the calculations on the
downloaded data against the `JPE96.DTA` file.  There were some very small
differences, because the NLS site has updated some of the data records.
Therefore, the data file saved here is very close to the contents of the
original `JPE96.DTA`.

Throughout, I have put the relevant Stata commands from the `RECREATE.DO` file into the comments, for comparison.

In [1]:
import re
from pathlib import Path

In [2]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

In [3]:
data_path = Path() / 'original'
df_export = pd.read_csv(data_path / 'neal_johnson_nls.csv')
df_export.head()

,R0000100,R0000300,R0000500,R0173600,R0214700,R0214800,R0410100,R0410300,R0614800,R0614900,...,R3728500,R3897100,R4006900,R4182300,R4295100,R4416800,R4418000,R4982800,R5079800,R5081000
0,1,9,58,5,3,2,9,58,-4,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,1,59,5,3,2,1,59,51,1,...,1263,22000,1,1,25000,1373,4,0,-4,7
2,3,8,61,5,3,2,-5,-5,51,0,...,153,4000,1,3,6000,581,4,0,-4,7
3,4,8,62,5,3,2,8,62,51,0,...,1528,30500,1,1,33000,1250,1,-5,-5,-5
4,5,7,59,1,3,1,7,59,51,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


In [4]:
# Number of cases from paper
assert len(df_export) == 12_686

These are the variable names from the `RECREATE.DO` file:

In [5]:
recreate_names = """\
id month79 year79 sid race sex79 month81 year81 code hgasvab
arith word comp numop math arithstd mathstd verbstd
afqt80 afqt89 sex82 class90 wage90 esr90 class91 wage91
earn91 esr91 class92 wage92 earn92 esr92 class93 earn93
wage93 esr93 earn94 wage94 esr94 
""".split()
recreate_names

['id',
 'month79',
 'year79',
 'sid',
 'race',
 'sex79',
 'month81',
 'year81',
 'code',
 'hgasvab',
 'arith',
 'word',
 'comp',
 'numop',
 'math',
 'arithstd',
 'mathstd',
 'verbstd',
 'afqt80',
 'afqt89',
 'sex82',
 'class90',
 'wage90',
 'esr90',
 'class91',
 'wage91',
 'earn91',
 'esr91',
 'class92',
 'wage92',
 'earn92',
 'esr92',
 'class93',
 'earn93',
 'wage93',
 'esr93',
 'earn94',
 'wage94',
 'esr94']

Here we record the pairs of variables names (NLS export name, `RECREATE.DO` name):

In [6]:
export_names = list(df_export)
pairs = list(zip(export_names, recreate_names))
pairs

[('R0000100', 'id'),
 ('R0000300', 'month79'),
 ('R0000500', 'year79'),
 ('R0173600', 'sid'),
 ('R0214700', 'race'),
 ('R0214800', 'sex79'),
 ('R0410100', 'month81'),
 ('R0410300', 'year81'),
 ('R0614800', 'code'),
 ('R0614900', 'hgasvab'),
 ('R0615100', 'arith'),
 ('R0615200', 'word'),
 ('R0615300', 'comp'),
 ('R0615400', 'numop'),
 ('R0615700', 'math'),
 ('R0618011', 'arithstd'),
 ('R0618017', 'mathstd'),
 ('R0618100', 'verbstd'),
 ('R0618200', 'afqt80'),
 ('R0618300', 'afqt89'),
 ('R0810200', 'sex82'),
 ('R3127500', 'class90'),
 ('R3127800', 'wage90'),
 ('R3401000', 'esr90'),
 ('R3523200', 'class91'),
 ('R3523500', 'wage91'),
 ('R3559000', 'earn91'),
 ('R3656400', 'esr91'),
 ('R3728200', 'class92'),
 ('R3728500', 'wage92'),
 ('R3897100', 'earn92'),
 ('R4006900', 'esr92'),
 ('R4182300', 'class93'),
 ('R4295100', 'earn93'),
 ('R4416800', 'wage93'),
 ('R4418000', 'esr93'),
 ('R4982800', 'earn94'),
 ('R5079800', 'wage94'),
 ('R5081000', 'esr94')]

Do these variable names make sense?   Check against the data dictionary
file from the export.

In [7]:
data_dict_lines = (data_path / 'neal_johnson_nls.sdf').read_text().splitlines()
print('\n'.join(data_dict_lines[:10]))



Reference
Number     Year    Variable Description                                                              Question Name  
----------------------------------------------------------------------------------------------------------------------
R00001.00  1979    IDENTIFICATION CODE                                                               CASEID         
R00003.00  1979    DATE OF BIRTH - MONTH                                                             Q1-3_A~M       
R00005.00  1979    DATE OF BIRTH - YEAR                                                              Q1-3_A~Y       
R01736.00  1979    SAMPLE IDENTIFICATION CODE                                                        SAMPLE_ID      
R02147.00  78SCRN  R'S RACIAL/ETHNIC COHORT FROM SCREENER                                            SAMPLE_RACE    


In [8]:
# Extract parts from line in data dictionary.
Q_RE = re.compile(r'(?P<no>R\d+\.\d+)\s+(?P<year>\d+\w*)\s+(?P<descr>.*?)\s+(?P<name>[A-Z0-9_\-~]+)\s*$')

We need to drop the first 5 lines, as they don't contain a variable definition.

In [9]:
line_dicts = []
for line in data_dict_lines[5:]:
    line_dicts.append(Q_RE.match(line).groupdict())
line_dicts[0]

{'no': 'R00001.00',
 'year': '1979',
 'descr': 'IDENTIFICATION CODE',
 'name': 'CASEID'}

Show the correspondence of recreate name, descriptions, year and export name.

In [10]:
fmt = "{:<10}{:<82}{:<8}{}"
header = fmt.format('RecName', 'Description', 'Year', 'Export Name')
print(header)
print('-' * len(header))
for vn, LD, en in zip(recreate_names, line_dicts, export_names):
    print(fmt.format(vn, LD['descr'], LD['year'], en))

RecName   Description                                                                       Year    Export Name
---------------------------------------------------------------------------------------------------------------
id        IDENTIFICATION CODE                                                               1979    R0000100
month79   DATE OF BIRTH - MONTH                                                             1979    R0000300
year79    DATE OF BIRTH - YEAR                                                              1979    R0000500
sid       SAMPLE IDENTIFICATION CODE                                                        1979    R0173600
race      R'S RACIAL/ETHNIC COHORT FROM SCREENER                                            78SCRN  R0214700
sex79     SEX OF R                                                                          1979    R0214800
month81   DATE OF BIRTH - MONTH                                                             1981    R0410100
year81    DAT

Titles look right.  Rename as above:

In [11]:
renames = {old_name: new_name
          for old_name, new_name in zip(export_names, recreate_names)}
df_renamed = df_export.rename(columns=renames)
df_renamed.head()

,id,month79,year79,sid,race,sex79,month81,year81,code,hgasvab,...,wage92,earn92,esr92,class93,earn93,wage93,esr93,earn94,wage94,esr94
0,1,9,58,5,3,2,9,58,-4,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,1,59,5,3,2,1,59,51,1,...,1263,22000,1,1,25000,1373,4,0,-4,7
2,3,8,61,5,3,2,-5,-5,51,0,...,153,4000,1,3,6000,581,4,0,-4,7
3,4,8,62,5,3,2,8,62,51,0,...,1528,30500,1,1,33000,1250,1,-5,-5,-5
4,5,7,59,1,3,1,7,59,51,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


Only the cross-section sample & the oversamples of blacks and hispanics.

Delete military & edw sample

In [12]:
sid = df_renamed['sid']
to_drop = (sid > 14) | (sid == 9) | (sid==12)
df = df_renamed[~to_drop].reset_index(drop=True)
df

,id,month79,year79,sid,race,sex79,month81,year81,code,hgasvab,...,wage92,earn92,esr92,class93,earn93,wage93,esr93,earn94,wage94,esr94
0,1,9,58,5,3,2,9,58,-4,-4,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,1,59,5,3,2,1,59,51,1,...,1263,22000,1,1,25000,1373,4,0,-4,7
2,3,8,61,5,3,2,-5,-5,51,0,...,153,4000,1,3,6000,581,4,0,-4,7
3,4,8,62,5,3,2,8,62,51,0,...,1528,30500,1,1,33000,1250,1,-5,-5,-5
4,5,7,59,1,3,1,7,59,51,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9758,12516,4,60,13,2,2,4,60,51,1,...,-5,-5,-5,1,-2,610,4,0,-4,7
9759,12517,7,62,10,2,1,7,62,51,0,...,660,15000,1,1,14400,693,1,14474,-4,1
9760,12518,3,64,13,2,2,3,64,51,0,...,752,2000,1,1,16000,695,1,-5,-5,-5
9761,12558,4,59,13,2,2,4,59,51,1,...,-4,0,4,-4,0,-4,3,0,-4,7


In [13]:
# The original 'JPE96.DTA` file had 9763 rows.
assert len(df) == 9763

Generate year of birth using data from two years.

Steve McClaskey of CHRR suggested this rule.

In [14]:
year81 = df['year81']
df['year'] = year81.where(year81 > 0, df['year79'])
# replace year=year81 if year81 > 0;
# replace year=year79 if year81 <=0;
df.loc[:, ['id', 'year81', 'year79', 'year']].tail()

,id,year81,year79,year
9758,12516,60,60,60
9759,12517,62,62,62
9760,12518,64,64,64
9761,12558,59,59,59
9762,12686,60,60,60


Set ASVAB scores to -9 if there are problems with the test.

We can use the math test as an indicator because respondents
either took them all or took none of them.  Further, the
altered testing codes apply to the exams as a whole.

Note add in 2004: Technically a few people took `arith` without taking
the other tests, but the code below still does what we want because
there are no positive math scores for people who have negatives on
other exams.

In [15]:
# replace math=-9      if code==53 | code==67 | math < 0;
# replace arith=-9     if math ==-9;
# replace comp=-9      if math ==-9;
# replace word=-9      if math ==-9;
# replace numop=-9     if math ==-9;
# replace arithstd=-9  if math ==-9;
# replace mathstd=-9   if math ==-9;
# replace verbstd=-9   if math ==-9;
# replace afqt80=-9    if math ==-9;
# replace afqt89=-9    if math ==-9;
code, math = df['code'], df['math']
bad_math = code.isin((53, 67)) | (math < 0)
test_cols =  ['math', 'arith', 'comp', 'word', 'numop',
              'arithstd', 'mathstd', 'verbstd', 'afqt80', 'afqt89']
df.loc[bad_math, test_cols] = -9

This is the scoring system for the 1989 version of the AFQT.

See the NLS user's guide - 1995, P. 52

In [16]:
# gen std89=(2*verbstd) + mathstd + arithstd;
std89 = 2 * df['verbstd'] + df['mathstd'] + df['arithstd']
# replace std89=-9     if math==-9;
df['std89'] = std89.where(~bad_math, -9)

Below we convert the scores to age standardized scores. Therefore, we need to mark bad birth years

In [17]:
# gen badyear=0;
# replace badyear=1 if year < 57 | year > 64;
year = df['year']
df['badyear'] = np.where((year < 57) | (year > 64), 1, 0)

Create age adjusted, standard scores for cases with valid data.

`std89res` is the variable that we used as an AFQT score.

In [18]:
# regress std89 y2 y3 y4 y5 y6 y7 y8 if math >= 0 & badyear==0;
valid_rows = (df['math'] >= 0) & (df['badyear'] == 0)
valid_df = df[valid_rows]

In [19]:
# regress std89 y2 y3 y4 y5 y6 y7 y8 if math >= 0 & badyear==0;
fitted = smf.ols("std89 ~ C(year)", data=valid_df).fit()
# predict std89res if math>=0 & badyear==0,rstandard;
# `,rstandard` gives "standardized residuals".
# See Stata rregresspostestimation.pdf and
# https://www.statsmodels.org/stable/generated/statsmodels.stats.outliers_influence.OLSInfluence
influence = fitted.get_influence()
std89res = influence.resid_studentized_internal
std89res

array([-1.15987296,  0.37758118,  0.47490962, ..., -0.94103678,
       -1.81929791,  0.91375552])

In [20]:
# replace std89res=-9 if std89res==.;
df['std89res'] = -9
df.loc[valid_rows, 'std89res'] = std89res.copy()

Here, we recreate our wage measure.

First we need to create an employment status variable.

* `status`=1 - resp. is working - valid entry for "class of worker."
* `status`=0 - resp. is not working - valid skip for "class of worker"
  and not in the military.
* `status`=-9 - the respondent is in the military or the respondent's
  wage data is missing due to coding problems.

In [21]:
# gen st90=-9;
# replace st90=1 if class90 > 0;
# replace st90=0 if class90==-4 & esr90~=8;
df['st90'] = -9
df.loc[df['class90'] > 0, 'st90'] = 1
df.loc[(df['class90'] == -4) & (df['esr90'] != 8), 'st90'] = 0

In [22]:
# gen st91=-9;
# replace st91=1 if class91 > 0;
# replace st91=0 if class91==-4 & esr91~=8;
df['st91'] = -9
df.loc[df['class91'] > 0, 'st91'] = 1
df.loc[(df['class91'] == -4) & (df['esr91'] != 8), 'st91'] = 0

Now edit the wage data.

If hourly wage exceeds \$75, then label it invalid.

Below, the same rule applies for wages < \$1

In [23]:
# replace wage90=-9 if wage90>7500;
# replace wage91=-9 if wage91>7500;
for col_name in 'wage90', 'wage91':
    df.loc[df[col_name] > 7500, col_name] = -9

Investigation against the `NPE96.DTA` data, showed that the latest download has some different values for the `wage90` variable.  Specifically these rows differed

| id | downloaded wage91 | NPE96.DTA wage91 |
| ----- | ---- | ---- |
| 3402  | 1053 | -4.0 |
| 4312  | 630  | -4.0 |
| 4841  | 1103 | -4.0 |
| 5855  | 460  | -4.0 |
| 5993  | 395  | -4.0 |
| 6111  | 235  | -4.0 |
| 6667  | 400  | -4.0 |
| 6678  | 142  | -4.0 |
| 8415  | 750  | -4.0 |
| 9287  | 1000 | -4.0 |
| 9899  | 1346 | -4.0 |
| 12001 | 1000 | -4.0 |

Create flags for resp. who belong in the sample for median regressions.  They must have a valid wage observation OR
`st** = 0` in at least one year.  We do not treat persons with bad wage observ or status=-9 as non-participants.

In [24]:
# gen flag=0;
# replace flag=1 if st90==0;
# replace flag=1 if st91==0;
df['flag'] = 0
df.loc[(df['st90'] == 0) | (df['st91'] == 0), 'flag'] = 1

`wage`=1 for the non-valid wage records & non-particpants. This is
okay because there is no-one who earns one cent per hour.

Thus `logwage=0` for both non-participants and coding errors.

In [25]:
# gen wage=1;   # if both wage obs are < 100
# replace wage=(wage91/1.04) if wage90 <= 100 & wage91 > 100;  # CPI
# replace wage=wage90 if wage91 <= 100 & wage90 > 100;
# replace wage=((wage90 + (wage91/1.04))/2) if wage90 > 100 & wage91 > 100;

def wage_apply(row):
    wage90, wage91 = row['wage90'], row['wage91']
    adj_91 = wage91 / 1.04  # CPI
    if wage90 <= 100 and wage91 > 100:
        return adj_91
    if wage91 <= 100 and wage90 > 100:
        return wage90
    if wage90 > 100 and wage91 > 100:
        return (wage90 + adj_91) / 2
    return 1
        
df['wage'] = df.apply(wage_apply, axis='columns')

In [26]:
# replace wage=1 if st90==0 & st91==0;   # mitigates coding error problem
df.loc[(df['st90'] == 0) & (df['st91'] == 0), 'wage'] = 1

In [27]:
# replace flag=1 if wage > 1;
df.loc[df['wage'] > 1, 'flag'] = 1

Because `st90` & `st91` were created from the class of worker variable, it is
possible to get `st90(91)==-9` & `wage90(91) > 1` for invalid skips on the
class variable.  However the `st90(91)==0` & `wage90(91) > 1` cases are more of
a puzzle.  Karima Nagee (CHRR) claims these (two) cases are likely coding
error.  Thus, in the case where both status variables are zero, we set the
`wage`=0.

On the other hand, there are approx. 380 people in the 1990 survey & a similar
number in 1991 who are a valid skip for wages but actually report a valid class
of worker.  These people are employed but gave bad or incomplete wage
information.

Create variables used in the paper.

In [28]:
# gen lwage=log(wage);
df['lwage'] = np.log(df['wage'])

Some of these rows are different from the original, because of data changes (above).  The rest are the same.

In [29]:
# gen age=90-year;
df['age'] = 90 - df['year']

In [30]:
# gen hisp=0;
# replace hisp=1 if race==1;
df['hisp'] = df['race'] == 1

In [31]:
# gen black=0;
# replace black=1 if race==2;
df['black'] = df['race'] == 2

In [32]:
# gen female=0;
# replace female=1 if sex82==2;
# replace female=1 if sex82 < 0 & sex79==2;
df['female'] = (df['sex82'] == 2) | ((df['sex82'] < 0) & (df['sex79'] == 2))

Further below, we will use these calculations in porting model formulae.

```
gen a1=std89res;
gen a2=a1*a1;
```

Create sample identifiers.

* `insamp1` : valid for OLS regressions.
* `insamp2` : valid for median (quartile) regressions.

In [33]:
# gen insamp1=0;
# replace insamp1=1 if lwage > 0 & math >= 0 & year > 61 & badyear==0;
df['insamp1'] = (df['lwage'] > 0) & (df['math'] >= 0) & (df['year'] > 61) & (df['badyear'] == 0)

Difference in `wage` field above means the resulting row values are all equal to original.

In [34]:
# gen insamp2=0;
# replace insamp2=1 if flag==1 & math >= 0  & year > 61 & badyear==0;
df['insamp2'] = (df['flag'] == 1) & (df['math'] >= 0) & (df['year'] > 61) & (df['badyear'] == 0)

These regressions replicate key columns of table 1 & table 4.

In [35]:
# regress lwage black hisp age if insamp1==1 & female==0;
# Column 1 of table 1.
s1_m_df = df[df['insamp1'] & ~df['female']]
s1_m_fit = smf.ols('lwage ~ black + hisp + age', data=s1_m_df).fit()
s1_m_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     33.58
Date:                Thu, 04 Aug 2022   Prob (F-statistic):           4.83e-21
Time:                        15:49:57   Log-Likelihood:                -986.98
No. Observations:                1594   AIC:                             1982.
Df Residuals:                    1590   BIC:                             2003.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         5.6061      0.379     14.791      0.000       4.863       6.350
black[T.True]    -0.2454      0.026     -9.419      0.000      -0.297      -0.194
hisp[T.True]     -0.1137      0.030     -3.756      0.000      -0.173      -0.054
age               0.0473      0.014      3.383      0.001       0.020       0.075
==============================================================================
Omnibus:                       47.460   Durbin-Watson:                   1.754
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               59.333
Skew:                           0.345   Prob(JB):                     1.31e-13
Kurtosis:                       3.646   Cond. No.                         912.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
# regress lwage black hisp age a1 a2 if insamp1==1 & female==0;
# Column 3 of table 1
s1_m_fit_ext = smf.ols('lwage ~ black + hisp + age + std89res + I(std89res ** 2)', data=s1_m_df).fit()
s1_m_fit_ext.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.166
Method:                 Least Squares   F-statistic:                     64.42
Date:                Thu, 04 Aug 2022   Prob (F-statistic):           2.44e-61
Time:                        15:49:57   Log-Likelihood:                -888.75
No. Observations:                1594   AIC:                             1790.
Df Residuals:                    1588   BIC:                             1822.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            5.7544      0.357     16.131      0.000       5.055       6.454
black[T.True]       -0.0720      0.027     -2.636      0.008      -0.126      -0.018
hisp[T.True]         0.0047      0.030      0.158      0.874      -0.054       0.063
age                  0.0395      0.013      3.001      0.003       0.014       0.065
std89res             0.1730      0.012     14.429      0.000       0.149       0.197
I(std89res ** 2)    -0.0132      0.011     -1.216      0.224      -0.035       0.008
==============================================================================
Omnibus:                       63.891   Durbin-Watson:                   1.816
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              109.933
Skew:                           0.320   Prob(JB):                     1.34e-24
Kurtosis:                       4.116   Cond. No.                         913.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [37]:
# qreg lwage black hisp age if insamp2==1 & female==0;
# Column 1 of table 4.
s2_m_df = df[df['insamp2'] & ~df['female']]
s2_m_qfit = smf.quantreg("lwage ~ black + hisp + age", data=s2_m_df).fit(q=0.5)
s2_m_qfit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                         QuantReg Regression Results                          
==============================================================================
Dep. Variable:                  lwage   Pseudo R-squared:              0.03228
Model:                       QuantReg   Bandwidth:                      0.1968
Method:                 Least Squares   Sparsity:                        1.204
Date:                Thu, 04 Aug 2022   No. Observations:                 1674
Time:                        15:49:57   Df Residuals:                     1670
                                        Df Model:                            3
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         5.0511      0.495     10.194      0.000       4.079       6.023
black[T.True]    -0.3560      0.034    -10.561      0.000      -0.422      -0.290
hisp[T.True]     -0.1810      0.040     -4.556      0.000      -0.259      -0.103
age               0.0675      0.018      3.690      0.000       0.032       0.103
=================================================================================
"""

In [38]:
# qreg lwage black hisp age a1 a2 if insamp2==1 & female==0;
# Column 2 of table 4.
s2_m_qfit_ext = smf.quantreg("lwage ~ black + hisp + age + std89res + I(std89res ** 2)", data=s2_m_df).fit(q=0.5)
s2_m_qfit_ext.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                         QuantReg Regression Results                          
==============================================================================
Dep. Variable:                  lwage   Pseudo R-squared:              0.07279
Model:                       QuantReg   Bandwidth:                      0.1770
Method:                 Least Squares   Sparsity:                        1.113
Date:                Thu, 04 Aug 2022   No. Observations:                 1674
Time:                        15:49:57   Df Residuals:                     1668
                                        Df Model:                            5
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            5.3011      0.458     11.575      0.000       4.403       6.199
black[T.True]       -0.1404      0.035     -4.025      0.000      -0.209      -0.072
hisp[T.True]        -0.0140      0.038     -0.364      0.716      -0.089       0.061
age                  0.0550      0.017      3.252      0.001       0.022       0.088
std89res             0.2060      0.015     13.398      0.000       0.176       0.236
I(std89res ** 2)     0.0102      0.014      0.728      0.467      -0.017       0.038
====================================================================================
"""

In [39]:
# regress lwage black hisp age if insamp1==1 & female==1;
# Column 4 of table 1
s1_f_df = df[df['insamp1'] & df['female']]
s1_f_fit = smf.ols('lwage ~ black + hisp + age', data=s1_f_df).fit()
s1_f_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     14.52
Date:                Thu, 04 Aug 2022   Prob (F-statistic):           2.56e-09
Time:                        15:49:57   Log-Likelihood:                -990.49
No. Observations:                1450   AIC:                             1989.
Df Residuals:                    1446   BIC:                             2010.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         6.3886      0.425     15.045      0.000       5.556       7.222
black[T.True]    -0.1874      0.029     -6.420      0.000      -0.245      -0.130
hisp[T.True]     -0.0266      0.033     -0.802      0.423      -0.092       0.038
age               0.0101      0.016      0.645      0.519      -0.021       0.041
==============================================================================
Omnibus:                       26.521   Durbin-Watson:                   1.783
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               46.764
Skew:                          -0.106   Prob(JB):                     7.00e-11
Kurtosis:                       3.854   Cond. No.                         914.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
# regress lwage black hisp age a1 a2 if insamp1==1 & female==1;
# Column 6 of table 1
s1_f_fit_ext = smf.ols('lwage ~ black + hisp + age + std89res + I(std89res ** 2)', data=s1_f_df).fit()
s1_f_fit_ext.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.163
Method:                 Least Squares   F-statistic:                     57.28
Date:                Thu, 04 Aug 2022   Prob (F-statistic):           1.83e-54
Time:                        15:49:57   Log-Likelihood:                -880.82
No. Observations:                1450   AIC:                             1774.
Df Residuals:                    1444   BIC:                             1805.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            5.9168      0.395     14.966      0.000       5.141       6.692
black[T.True]        0.0329      0.031      1.072      0.284      -0.027       0.093
hisp[T.True]         0.1446      0.033      4.413      0.000       0.080       0.209
age                  0.0228      0.015      1.568      0.117      -0.006       0.051
std89res             0.2278      0.015     14.728      0.000       0.197       0.258
I(std89res ** 2)     0.0119      0.014      0.872      0.383      -0.015       0.039
==============================================================================
Omnibus:                       69.536   Durbin-Watson:                   1.804
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              203.143
Skew:                          -0.169   Prob(JB):                     7.73e-45
Kurtosis:                       4.802   Cond. No.                         918.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The keep statement gives the variables used in our basic wage regressions plus
sample identifiers and related tests score items.  Note: `afqt80` & `afqt89`
are percentile scores.

In [41]:
to_keep = """\
id sid female black hisp year age insamp1 insamp2 badyear
wage90 wage91 st90 st91 flag lwage
afqt80 afqt89 std89 std89res math arith word comp numop""".split()
final_df = df.loc[:, to_keep]
final_df.head()

,id,sid,female,black,hisp,year,age,insamp1,insamp2,badyear,...,lwage,afqt80,afqt89,std89,std89res,math,arith,word,comp,numop
0,1,5,True,False,False,58,32,False,False,0,...,0.000000,-9,-9,-9,-9.000000,-9,-9,-9,-9,-9
1,2,5,True,False,False,59,31,False,False,0,...,6.981828,12,9,144,-1.159873,6,8,15,6,29
2,3,5,True,False,False,61,29,False,False,0,...,6.455199,51,46,200,0.377581,13,13,29,12,46
3,4,5,True,False,False,62,28,True,True,0,...,7.264030,62,48,202,0.474910,10,18,29,12,46
4,5,1,False,False,False,59,31,False,False,0,...,0.000000,90,99,254,1.741597,25,30,35,13,35


In [42]:
# Convert bool to int for greater portability of the written CSV file.
for col_name in final_df:
    if final_df[col_name].dtype == bool:
        final_df[col_name] = final_df[col_name].astype(int)
final_df.dtypes

id            int64
sid           int64
female        int64
black         int64
hisp          int64
year          int64
age           int64
insamp1       int64
insamp2       int64
badyear       int64
wage90        int64
wage91        int64
st90          int64
st91          int64
flag          int64
lwage       float64
afqt80        int64
afqt89        int64
std89         int64
std89res    float64
math          int64
arith         int64
word          int64
comp          int64
numop         int64
dtype: object

Write file.  Check and read back.

In [43]:
out_fname = Path() / 'processed' / 'race_wages.csv'
final_df.to_csv(out_fname, index=None)

In [44]:
pd.read_csv(out_fname).head()

,id,sid,female,black,hisp,year,age,insamp1,insamp2,badyear,...,lwage,afqt80,afqt89,std89,std89res,math,arith,word,comp,numop
0,1,5,1,0,0,58,32,0,0,0,...,0.000000,-9,-9,-9,-9.000000,-9,-9,-9,-9,-9
1,2,5,1,0,0,59,31,0,0,0,...,6.981828,12,9,144,-1.159873,6,8,15,6,29
2,3,5,1,0,0,61,29,0,0,0,...,6.455199,51,46,200,0.377581,13,13,29,12,46
3,4,5,1,0,0,62,28,1,1,0,...,7.264030,62,48,202,0.474910,10,18,29,12,46
4,5,1,0,0,0,59,31,0,0,0,...,0.000000,90,99,254,1.741597,25,30,35,13,35
